### imports

In [7]:
import json
import requests
from bs4 import BeautifulSoup

2. Sending HTTP request to access first page with opinions

In [ ]:
url = "https://www.ceneo.pl/84514582#tab=reviews"
response = requests.get(url)

3.Checking the code of HTTP response

In [ ]:
response.status_code

4.Parse the HTML code of first page with opinions

In [9]:
page_dom = BeautifulSoup( response.text, 'html.parser')


5.Extract required data from parsed code

In [ ]:
opinions = page_dom.select('div.js_product-review:not(.user-post--highlight)')
for opinion in opinions:
    opinion_id = opinion['data-entry-id']
    author = opinion.select_one('span.user-post__author-name').get_text()
    try:
        reccommendation = opinion.select_one('span.user-post__author-recomendation > em').get_text()
    except AttributeError: 
        reccommendation = None
    stars = opinion.select_one('span.user-post__score-count').get_text()
    content = opinion.select_one('div.user-post__text').get_text()
    pros = [p.get_text()for p in opinion.select('div.review-feature__item--positive')]
    cons = [c.get_text() for c in opinion.select('div.review-feature__item--negative')]
    vote_yes = opinion.select_one('button.vote-yes')['data-total-vote']
    vote_no = opinion.select_one('button.vote-no')['data-total-vote']
    published = opinion.select_one('span.user-post__published > time:nth-child(1)')["datetime"]
    try:
        purchased = opinion.select_one('span.user-post__published > time:nth-child(2)')["datetime"]
    except TypeError: 
        purchased = None
    print(opinion_id, author, reccommendation, stars, content, pros, cons, vote_yes, vote_no, published, purchased, sep='\n')

6. If there are more pages, move to the next page and repeat steps 2-6 for it

In [ ]:
try:
    next_page = 'https://www.ceneo.pl' + page_dom.select_one('a.pagination__next')['href']
except TypeError: 
    next_page = None

In [52]:
next_page = "https://www.ceneo.pl/84514582#tab=reviews"
all_opinions = {}
while next_page:
    response = requests.get(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup( response.text, 'html.parser')
        opinions = page_dom.select('div.js_product-review:not(.user-post--highlight)')
        for opinion in opinions:
            single_opinion = {}
            single_opinion['opinion_id'] = opinion['data-entry-id']
            single_opinion['author'] = opinion.select_one('span.user-post__author-name').get_text()
            try:
                single_opinion['reccommendation'] = opinion.select_one('span.user-post__author-recomendation > em').get_text()
            except AttributeError: 
                single_opinion['reccommendation'] = None
            single_opinion['stars'] = opinion.select_one('span.user-post__score-count').get_text()
            single_opinion['content'] = opinion.select_one('div.user-post__text').get_text()
            single_opinion['pros'] = [p.get_text()for p in opinion.select('div.review-feature__item--positive')]
            single_opinion['cons'] = [c.get_text() for c in opinion.select('div.review-feature__item--negative')]
            single_opinion['vote_yes'] = opinion.select_one('button.vote-yes')['data-total-vote']
            single_opinion['vote_no'] = opinion.select_one('button.vote-no')['data-total-vote']
            single_opinion['published'] = opinion.select_one('span.user-post__published > time:nth-child(1)')["datetime"]
            try:
                single_opinion['purchased'] = opinion.select_one('span.user-post__published > time:nth-child(2)')["datetime"]
            except TypeError: 
                single_opinion['purchased'] = None
            all_opinions.append(single_opinion)
        try:
            next_page = 'https://www.ceneo.pl' + page_dom.select_one('a.pagination__next')['href']
        except TypeError: 
            next_page = None
        